<a href="https://colab.research.google.com/github/rajenheg10/aiRepo/blob/main/sp500HeatMap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import plotly.graph_objects as go
import pandas as pd
import numpy as np


url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
table = pd.read_html(url, header=0)
df = table[0]
tickers = df['Symbol'].tolist()
sectors = df.set_index('Symbol')['GICS Sector'].to_dict()

# Limit to 100 for demo to avoid timeouts (use full list for real use)
tickers_sample = tickers[:100]

data = yf.download(tickers_sample, period="1d", interval="1d", group_by='ticker', threads=True)

price_changes = {}
for ticker in tickers_sample:
    try:
        open_price = data[ticker]['Open'].iloc[0]
        close_price = data[ticker]['Close'].iloc[0]
        change = ((close_price - open_price) / open_price) * 100
        price_changes[ticker] = round(change, 2)
    except Exception:
        price_changes[ticker] = None

tickers_ordered = list(price_changes.keys())
values = [price_changes[t] for t in tickers_ordered]
labels = [f"{t}<br>{sectors.get(t, '')}<br>{v:.2f}%" if v is not None else f"{t}<br>--"
          for t, v in zip(tickers_ordered, values)]

grid_size = int(np.ceil(np.sqrt(len(labels))))
labels += [''] * (grid_size ** 2 - len(labels))
values += [None] * (grid_size ** 2 - len(values))

labels_grid = np.array(labels).reshape(grid_size, grid_size)
values_grid = np.array(values).reshape(grid_size, grid_size)


fig = go.Figure(data=go.Heatmap(
    z=values_grid,
    text=labels_grid,
    hoverinfo="text",
    colorscale='RdYlGn',
    zmin=-5,
    zmax=5,
    showscale=True,
    colorbar=dict(title="Change %")
))

fig.update_layout(
    title='S&P 500 Heatmap (Top 100 Tickers)',
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis=dict(showticklabels=False),
    yaxis=dict(showticklabels=False),
    autosize=False,
    width=800,
    height=800,
)

fig.show()




[*********************100%***********************]  100 of 100 completed
ERROR:yfinance:
2 Failed downloads:
ERROR:yfinance:['BF.B']: YFPricesMissingError('possibly delisted; no price data found  (period=1d)')
ERROR:yfinance:['BRK.B']: YFPricesMissingError('possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")')
